In order to access Osparc you will need to generate Osparc [tokens](https://docs.osparc.io/#/docs/platform_introduction/user_setup/security_details?id=generating-o%c2%b2s%c2%b2parc-tokens) and expose these to your environment in the form of environment variables. More precisely you will have to expose `O2SPARC_HOST`, `O2SPARC_USERNAME` and `O2SPARC_PASSWORD`.

In [ ]:
# import the necessary packages
import importlib
if importlib.util.find_spec('sparc.client') is not None:
  ! pip install sparc.client
if importlib.util.find_spec('tqdm') is not None:
  ! pip install tqdm
from pprint import pprint
import os
from tqdm import tqdm
from sparc.client import SparcClient
from sparc.client.services.o2sparc import (
  O2SparcService,
  O2SparcSolver
)
from time import sleep
from pathlib import Path
from tempfile import TemporaryDirectory

assert "O2SPARC_HOST" in os.environ, "O2SPARC_HOST must be exposed as an environment variable"
assert "O2SPARC_USERNAME" in os.environ, "O2SPARC_USERNAME must be exposed as an environment variable"
assert "O2SPARC_PASSWORD" in os.environ, "O2SPARC_PASSWORD must be exposed as an environment variable"


Access to the computational services provided by the `o2sparc` submodule, is provided via `O2SparcService`, an instance of which can be creates as follows:

In [18]:

client = SparcClient(connect=False, config_file='config.ini')
o2sparc: O2SparcService = client.o2sparc


As a sanity check you can now contact the Osparc server to check that you are logged in as the correct user:

In [ ]:
print(o2sparc.get_profile())
# foo@bar.com

Now to the fun part :). In the following cell we specify a job (or a "computation") which we submit to a solver (or "computational resource") on the Osparc platform.

In [ ]:
with TemporaryDirectory() as tmp_dir:
  input_file: Path = Path(tmp_dir) / "input_file.txt"
  input_file.write_text("3")

  job: dict = {
    "input_3": 0,
    "input_2": 3.0,
    "input_1": input_file
  }

  solver: O2SparcSolver = o2sparc.get_solver(solver_key="simcore/services/comp/itis/sleeper",solver_version="2.0.2")

  job_id = solver.submit_job(job)

  pbar = tqdm(total=1.0)
  progress: float = 0
  while not solver.job_done(job_id):
    sleep(1)
    if solver.get_job_progress(job_id) > progress:
      pbar.update(solver.get_job_progress(job_id) - progress)
      progress = solver.get_job_progress(job_id)

  print("job outputs:")
  for output_name, result in solver.get_results(job_id).items():
    if isinstance(result,Path):
      print(f"{output_name}: {Path(result).read_text()}")
    else:
      print(f"{output_name}: {result}")


  print("job log:")
  log_dir: TemporaryDirectory = solver.get_job_log(job_id)
  for elm in Path(log_dir.name).rglob("*"):
    if elm.is_file():
      print(elm.read_text())
      

Let's break this into pieces. 
First we specify the job/input to our solver. A job for a `O2SparcSolver` is nothing but a dictionary whose entries are one of the following types: `str`, `int`, `float` or `pathlib.Path`. Only `pathlib.Path` objects can be passed which point to existing files, like out `input_1` above.

```python
  input_file: Path = Path(tmp_dir) / "input_file.txt"
  input_file.write_text("3")

  job: dict = {
    "input_3": 0,
    "input_2": 3.0,
    "input_1": input_file
  }
```

Second, we get the solver/computational resource we want to use by specifying its identifier and version. Once we have our solver we can submit the job to it.

```python
  solver: O2SparcSolver = o2sparc.get_solver(solver_key="simcore/services/comp/itis/sleeper",solver_version="2.0.2")

  job_id = solver.submit_job(job)
```

Next, we need to wait for the job to be solved

```python
 pbar = tqdm(total=1.0)
  progress: float = 0
  while not solver.job_done(job_id):
    sleep(1)
    if solver.get_job_progress(job_id) > progress:
      pbar.update(solver.get_job_progress(job_id) - progress)
      progress = solver.get_job_progress(job_id)
```

Once the solver is done performing its computation we can get the results it produces

```python
  print("job outputs:")
  for output_name, result in solver.get_results(job_id).items():
    if isinstance(result,Path):
      print(f"{output_name}: {Path(result).read_text()}")
    else:
      print(f"{output_name}: {result}")
```

as well as the log it has produces

```python
  print("job log:")
  log_dir: TemporaryDirectory = solver.get_job_log(job_id)
  for elm in Path(log_dir.name).rglob("*"):
    if elm.is_file():
      print(elm.read_text())
```

Getting the log can be particularly useful for determining issues encountered by the solver. E.g. if the specified `job` is invalid, so that the solver cannot use it, the log will typically indicate this.